## Imports

In [27]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os

import statsmodels.api as sm
from scipy import signal
from scipy import stats
import pandas as pd

In [28]:
enso_ts = pd.read_csv('./enso_ond.csv')
jet_ts = pd.read_csv('./jet_ond.csv')
spv_ts = pd.read_csv('./spv_ond.csv')

In [29]:
# Standardise the data (zero mean, unit variance)

ENSO = (enso_ts['enso'] - np.mean(enso_ts['enso'])) / np.std(enso_ts['enso'])
JET = (jet_ts['jet'] - np.mean(jet_ts['jet'])) / np.std(jet_ts['jet'])
SPV = (spv_ts['spv'] - np.mean(spv_ts['spv'])) / np.std(spv_ts['spv'])

from scipy import signal

# Assuming ENSO, JET, SPV are your standardized NumPy arrays from the previous step

# Detrending the standardized data
ENSO_detrended = signal.detrend(ENSO)
JET_detrended = signal.detrend(JET)
SPV_detrended = signal.detrend(SPV)

# If ENSO, JET, SPV are pandas Series from the DataFrames, you can convert them to NumPy arrays as follows:
#ENSO_detrended = signal.detrend(ENSO.values)
#ET_detrended = signal.detrend(JET.values)
#SPV_detrended = signal.detrend(SPV.values)

# Example to print and verify the detrended data
print("Detrended ENSO sample:", ENSO_detrended[:5])
print("Detrended JET sample:", JET_detrended[:5])
print("Detrended SPV sample:", SPV_detrended[:5])


Detrended ENSO sample: [-0.65886546  0.84741166 -0.07658269  0.64324851 -0.811211  ]
Detrended JET sample: [-0.14788254 -0.98349973 -0.72129053 -0.99989776  0.45152022]
Detrended SPV sample: [2.27982732 0.69088251 0.01539778 0.25337314 2.40131051]


In [30]:
# Detrend the data (remove linear trend) . You can use signal.detrend()

In [31]:
# plot detrended data

import matplotlib.pyplot as plt

years = enso_ts['year'].values  # Extracting the years from any of the loaded DataFrames

# Creating the plots
plt.figure(figsize=(14, 8))

# Plotting ENSO detrended data
plt.subplot(3, 1, 1)  # 3 rows, 1 column, 1st subplot
plt.plot(years, ENSO_detrended, label='ENSO Detrended')
plt.title('Detrended ENSO Data')
plt.ylabel('Standardized Anomaly')
plt.legend()

# Plotting JET detrended data
plt.subplot(3, 1, 2)  # 3 rows, 1 column, 2nd subplot
plt.plot(years, JET_detrended, label='JET Detrended', color='orange')
plt.title('Detrended JET Data')
plt.ylabel('Standardized Anomaly')
plt.legend()

# Plotting SPV detrended data
plt.subplot(3, 1, 3)  # 3 rows, 1 column, 3rd subplot
plt.plot(years, SPV_detrended, label='SPV Detrended', color='green')
plt.title('Detrended SPV Data')
plt.ylabel('Standardized Anomaly')
plt.xlabel('Year')
plt.legend()

plt.tight_layout()
plt.show()

### Part 3

In [32]:
# for correlation you can use stats.pearsonr()
#======================================================================================
# A. Calculate the Pearson Correlation of MED and DK: is it significant at 0.05 p-value?
#### Calculate the Pearson Correlation of JET and ENSO: is it significant at 0.05 p-value?
#======================================================================================

ENSO = (enso_ts['enso'] - np.mean(enso_ts['enso'])) / np.std(enso_ts['enso'])
JET = (jet_ts['jet'] - np.mean(jet_ts['jet'])) / np.std(jet_ts['jet'])
SPV = (spv_ts['spv'] - np.mean(spv_ts['spv'])) / np.std(spv_ts['spv'])

#Xi = ENSO
#Yi = JET

import scipy.stats as stats

# Define variables
Xi = JET_detrended
Yi = ENSO_detrended
Zi = SPV_detrended

# Compute Pearson correlation and p-values
r_jet_enso, p_jet_enso = stats.pearsonr(Xi, Yi)  # JET vs ENSO
r_jet_spv, p_jet_spv = stats.pearsonr(Xi, Zi)  # JET vs SPV
r_enso_spv, p_enso_spv = stats.pearsonr(Yi, Zi)  # ENSO vs SPV

# Print results
print("Correlation and p-values:")
print(f"JET & ENSO: r = {round(r_jet_enso, 3)}, p = {round(p_jet_enso, 3)}")
print(f"JET & SPV: r = {round(r_jet_spv, 3)}, p = {round(p_jet_spv, 3)}")
print(f"ENSO & SPV: r = {round(r_enso_spv, 3)}, p = {round(p_enso_spv, 3)}")


Correlation and p-values:
JET & ENSO: r = -0.173, p = 0.159
JET & SPV: r = 0.473, p = 0.0
ENSO & SPV: r = -0.267, p = 0.028


### Parts 4 & 5

In [33]:
# Define variables for each regression
X_enso_spv = np.stack([ENSO_detrended, SPV_detrended]).T  # Controls: ENSO, SPV
Y_jet = JET_detrended  # Dependent: JET

X_enso_jet = np.stack([ENSO_detrended, JET_detrended]).T  # Controls: ENSO, JET
Y_spv = SPV_detrended  # Dependent: SPV

X_spv_enso = np.stack([SPV_detrended, ENSO_detrended]).T  # Controls: SPV, ENSO
Y_jet_spv = JET_detrended  # Dependent: JET

# Perform OLS regressions
model_jet = sm.OLS(Y_jet, X_enso_spv)
results_jet = model_jet.fit()
causal_effect_enso_on_jet = results_jet.params[0]  # ENSO → JET (controlling for SPV)

model_spv = sm.OLS(Y_spv, X_enso_jet)
results_spv = model_spv.fit()
causal_effect_enso_on_spv = results_spv.params[0]  # ENSO → SPV (controlling for JET)

model_jet_spv = sm.OLS(Y_jet_spv, X_spv_enso)
results_jet_spv = model_jet_spv.fit()
causal_effect_spv_on_jet = results_jet_spv.params[0]  # SPV → JET (controlling for ENSO)

# Print results in the requested format
print("Causal Effects:")
print(f"ENSO → JET controlling for SPV: {round(causal_effect_enso_on_jet, 4)}")
print(f"ENSO → SPV controlling for JET: {round(causal_effect_enso_on_spv, 4)}")
print(f"SPV → JET controlling for ENSO: {round(causal_effect_spv_on_jet, 4)}")

# Cleanup to avoid variable clashes
del model_jet, results_jet, model_spv, results_spv, model_jet_spv, results_jet_spv
del X_enso_spv, Y_jet, X_enso_jet, Y_spv, X_spv_enso, Y_jet_spv


Causal Effects:
ENSO → JET controlling for SPV: -0.0416
ENSO → SPV controlling for JET: -0.1886
SPV → JET controlling for ENSO: 0.3877


### Part 6 

In [35]:
X = ENSO_detrended
Y = JET_detrended 
model = sm.OLS(Y,X)
results = model.fit()
ce_enso_jet = results.params[0]

del(model,results,X,Y)

print("The causal effect of NAO on MED  is ", round(ce_enso_jet,4))

The causal effect of NAO on MED  is  -0.1439


### Part 7

In [37]:
import scipy.stats as stats
import pandas as pd
import numpy as np

# Load data
enso_ts = pd.read_csv('./enso_ond.csv')
jet_ts = pd.read_csv('./jet_ond.csv')
spv_ts = pd.read_csv('./spv_ond.csv')

# Create filtered copies (1978-1998) to avoid modifying the original data
enso_ts_filtered = enso_ts[(enso_ts['year'] >= 1978) & (enso_ts['year'] <= 1998)].copy()
jet_ts_filtered = jet_ts[(jet_ts['year'] >= 1978) & (jet_ts['year'] <= 1998)].copy()
spv_ts_filtered = spv_ts[(spv_ts['year'] >= 1978) & (spv_ts['year'] <= 1998)].copy()

# Extract values for correlation calculations
ENSO_filtered = enso_ts_filtered['enso'].values
JET_filtered = jet_ts_filtered['jet'].values
SPV_filtered = spv_ts_filtered['spv'].values

# Compute Pearson correlation and p-values
r_jet_enso, p_jet_enso = stats.pearsonr(JET_filtered, ENSO_filtered)  # JET vs ENSO
r_jet_spv, p_jet_spv = stats.pearsonr(JET_filtered, SPV_filtered)  # JET vs SPV
r_enso_spv, p_enso_spv = stats.pearsonr(ENSO_filtered, SPV_filtered)  # ENSO vs SPV

# Print results
print("Pearson Correlations and p-values (1978-1998):")
print(f"JET & ENSO: r = {round(r_jet_enso, 3)}, p = {round(p_jet_enso, 3)}")
print(f"JET & SPV: r = {round(r_jet_spv, 3)}, p = {round(p_jet_spv, 3)}")
print(f"ENSO & SPV: r = {round(r_enso_spv, 3)}, p = {round(p_enso_spv, 3)}")


Pearson Correlations and p-values (1978-1998):
JET & ENSO: r = -0.448, p = 0.042
JET & SPV: r = 0.584, p = 0.005
ENSO & SPV: r = -0.319, p = 0.158
